In [47]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
# Dependencies
import numpy as np
import pandas as pd
import math
import decimal
import requests
import time
from pprint import pprint 
from matplotlib import pyplot as plt
import os
import json
import urllib.request
from sqlalchemy import create_engine

In [49]:
census_track_newyork_data = "output_data/census_track_newyork_data.csv"
census_track_newyork_data_df = pd.read_csv(
    census_track_newyork_data)
census_track_newyork_data_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,census_block
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,10/19/18,0.21,6,365,360470504003000
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,5/21/19,0.38,2,355,360610084001000
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365,360610208002002
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,7/5/19,4.64,1,194,360470231002003
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,11/19/18,0.10,1,0,360610184003000


In [50]:
census_track_newyork_data_list = ["id", "name", "host_id", "host_name", "neighbourhood_group","neighbourhood", "latitude", "longitude", "room_type", "price", "census_block"]
new_census_track_newyork_data_df = census_track_newyork_data_df[census_track_newyork_data_list].copy()

new_census_track_newyork_data_df.head()


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,census_block
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,360470504003000
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,360610084001000
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,360610208002002
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,360470231002003
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,360610184003000


In [51]:
#type(new_census_track_newyork_data_df.census_block[0])
new_census_track_newyork_data_df['census_block_str'] = new_census_track_newyork_data_df.census_block.astype(str)
new_census_track_newyork_data_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,census_block,census_block_str
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,360470504003000,360470504003000
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,360610084001000,360610084001000
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,360610208002002,360610208002002
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,360470231002003,360470231002003
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,360610184003000,360610184003000


In [52]:
type(new_census_track_newyork_data_df.census_block[0])

numpy.int64

In [53]:
#36005000100
new_census_track_newyork_data_df['censustract'] = new_census_track_newyork_data_df.census_block_str.str[:11] 
new_census_track_newyork_data_df.head()


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,census_block,census_block_str,censustract
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,360470504003000,360470504003000,36047050400
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,360610084001000,360610084001000,36061008400
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,360610208002002,360610208002002,36061020800
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,360470231002003,360470231002003,36047023100
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,360610184003000,360610184003000,36061018400


In [54]:
census_track_newyork_airbnb_data_list = ["id", "name", "host_id", "host_name", "neighbourhood_group","neighbourhood", "latitude", "longitude", "room_type", "price", "censustract"]
census_track_newyork_airbnb_data_df = new_census_track_newyork_data_df[census_track_newyork_airbnb_data_list].copy()

census_track_newyork_airbnb_data_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,censustract
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,36047050400
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,36061008400
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,36061020800
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,36047023100
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,36061018400


In [55]:
census_track_newyork_airbnb_data_df.columns

#new_census_track_newyork_data_df = new_census_track_newyork_data_df.rename(columns={'County':'county', 'Borough':'borough', 'TotalPop':'totalpop', 'Hispanic':'hispanic', 'White':'white','Black':'black', 'Native':'native', 'Asian':'asian', 'Income':'income', 'IncomePerCap':'incomepercap', 'Poverty':'poverty','Employed':'employed', 'Unemployment':'unemployment'})

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'censustract'],
      dtype='object')

In [60]:
type(census_track_newyork_airbnb_data_df['censustract'][0])

str

In [62]:
census_track_newyork_airbnb_data_df.censustract = census_track_newyork_airbnb_data_df.censustract.astype(int)
census_track_newyork_airbnb_data_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,censustract
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,36047050400
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,36061008400
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,36061020800
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,36047023100
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,36061018400


In [63]:
type(census_track_newyork_airbnb_data_df['censustract'][0])

numpy.int64

In [66]:
rds_connection_string = "arun:postgres@localhost:5442/census_airbnb_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['census_data', 'airbnb_data']

In [67]:
# Load DataFrames into database
census_track_newyork_airbnb_data_df.to_sql(name='airbnb_data', con=engine, if_exists='append', index=False)